# Near Earth Object DtLab Demo

The DtLab deployment at https://somind.tech has been loaded with NASA Near Earth Object data.  The data was posted in its raw format downloaded from [NASA](https://api.nasa.gov/) into the [dtlab-ingest](https://somind.tech/dtlab-alligator/doc/dtlab-ingest/) service that in turn extracts DtLab-ready observations and posts then to DtLab - automatically creating a tree of actor-based digital twins for the asteroids (objects) orbiting Earth (orbiting_body).

```
NASA API ==> DtLab Ingest Service ==> DtLab API ==> Earth DT ==> Asteroid DT 1
                                                             ==> Asteroid DT 2
                                                             ==> Asteroid DT n
                                                                                               ```
                                              

## Setup

Import the `requests` lib to interact with the DtLab API.  Import `json` to pretty-print the json results.

The notebook server is co-located with the DtLab system.  While the externally visible service for DtLab is securied with TLS and Auth0-issued tokens - no extra auth is needed if you use the internally visible Kubernetes service name and port.  Set the base of the cluster's internally visable service URL in the `BASE_URL` variable.

In [1]:
import requests
import json
headers = {
  'Accept': 'application/json'
}
BASE_URL = "http://dtlab-scala-alligator.default:8081/dtlab-alligator/"

## Look at Type Definitions

Retrieve the `orbiting_body` type and see that it has children of type `object`.

In [2]:
response = requests.get(BASE_URL + 'type/orbiting_body', headers = headers)
print(json.dumps(response.json(), indent=2))

{
  "children": [
    "object"
  ],
  "created": "2020-11-15T14:56:54.009Z",
  "name": "orbiting_body"
}


Retrieve the `object` type definition and see that it has a number of properties related to the location and speed of the asteroid.

In [3]:
response = requests.get(BASE_URL + 'type/object', headers = headers)
print(json.dumps(response.json(), indent=2))

{
  "created": "2020-11-15T14:56:51.739Z",
  "name": "object",
  "props": [
    "absolute_magnitude",
    "diameter_min",
    "diameter_max",
    "miles_per_hour",
    "distance_in_miles"
  ]
}


# Peek at the State of an Actor 

Retreive the state of a digital twin of one of the asteroids.

See that it has 5 different values it is keeping track of and that it knows the time the most recent measurement was logged.

The `idx` identifier relates to the zero-indexed position of the property in the type definition.  `idx` of 3 means this is the `miles_per_hour` reading if you consult the `object` type definition above.

In [4]:
response = requests.get(BASE_URL + 'actor/orbiting_body/earth/object/3799865', headers = headers)
print(json.dumps(response.json(), indent=2))

[
  {
    "datetime": "2020-11-15T17:14:37.331Z",
    "idx": 0,
    "value": 34.282
  },
  {
    "datetime": "2020-11-15T17:14:37.332Z",
    "idx": 1,
    "value": 0.3699601753
  },
  {
    "datetime": "2020-11-15T17:14:37.332Z",
    "idx": 2,
    "value": 0.827256101
  },
  {
    "datetime": "2020-11-15T17:14:37.333Z",
    "idx": 3,
    "value": 15096.9310252428
  },
  {
    "datetime": "2020-11-15T17:14:37.334Z",
    "idx": 4,
    "value": 10013.4132206738
  }
]


The `idx` identifier is tedious to understand - lets use the `format` query param to get a meaningful name with each value.

In [5]:
response = requests.get(BASE_URL + 'actor/orbiting_body/earth/object/3799865?format=pathed', headers = headers)
print(json.dumps(response.json(), indent=2))

[
  {
    "datetime": "2020-11-15T17:14:37.331Z",
    "name": "orbiting_body.earth.object.3799865.absolute_magnitude",
    "value": 34.282
  },
  {
    "datetime": "2020-11-15T17:14:37.332Z",
    "name": "orbiting_body.earth.object.3799865.diameter_min",
    "value": 0.3699601753
  },
  {
    "datetime": "2020-11-15T17:14:37.332Z",
    "name": "orbiting_body.earth.object.3799865.diameter_max",
    "value": 0.827256101
  },
  {
    "datetime": "2020-11-15T17:14:37.333Z",
    "name": "orbiting_body.earth.object.3799865.miles_per_hour",
    "value": 15096.9310252428
  },
  {
    "datetime": "2020-11-15T17:14:37.334Z",
    "name": "orbiting_body.earth.object.3799865.distance_in_miles",
    "value": 10013.4132206738
  }
]


The long path as name is great for reading or for processing large collections of objects together but if your application context is just processing one instance of a DT at a time then the short name is easier to consume.  Try the query param for format `named`.

In [6]:
response = requests.get(BASE_URL + 'actor/orbiting_body/earth/object/3799865?format=named', headers = headers)
print(json.dumps(response.json(), indent=2))

[
  {
    "datetime": "2020-11-15T17:14:37.331Z",
    "name": "absolute_magnitude",
    "value": 34.282
  },
  {
    "datetime": "2020-11-15T17:14:37.332Z",
    "name": "diameter_min",
    "value": 0.3699601753
  },
  {
    "datetime": "2020-11-15T17:14:37.332Z",
    "name": "diameter_max",
    "value": 0.827256101
  },
  {
    "datetime": "2020-11-15T17:14:37.333Z",
    "name": "miles_per_hour",
    "value": 15096.9310252428
  },
  {
    "datetime": "2020-11-15T17:14:37.334Z",
    "name": "distance_in_miles",
    "value": 10013.4132206738
  }
]
